¿Cuántos términos hay en cada corpus?

Contando palabras repetidas:

In [19]:
import os

def palabras_totales(archivo):
    total_palabras = 0
    with open(archivo, 'r', encoding='utf-8') as f:
                contenido = f.read()
                palabras = contenido.split()
                num_palabras = len(palabras)
                total_palabras += num_palabras
                print(f"Archivo: {archivo}, Palabras: {num_palabras}")
    return total_palabras

archivo = r"C:\Users\ELI\Documents\RecInformación\ir25a-main\data\01_corpus_turismo.txt"
total = palabras_totales(archivo)
print("Total de palabras en los archivos .txt:", total)



Archivo: C:\Users\ELI\Documents\RecInformación\ir25a-main\data\01_corpus_turismo.txt, Palabras: 240
Total de palabras en los archivos .txt: 240


Sin palabras repetidas:

In [18]:
import os

def contar_palabras_unicas(archivo):
    palabras_unicas = set()
    with open(archivo, 'r', encoding='utf-8') as f:
                contenido = f.read()
                palabras = contenido.split()
                palabras_unicas.update(palabras)
                print(f"Archivo: {archivo}, Palabras únicas: {len(set(palabras))}")
    return len(palabras_unicas)

archivo = r"C:\Users\ELI\Documents\RecInformación\ir25a-main\data\01_corpus_turismo.txt"
total_unicas = contar_palabras_unicas(archivo)
print("Total de palabras únicas en los archivos .txt:", total_unicas)


Archivo: C:\Users\ELI\Documents\RecInformación\ir25a-main\data\01_corpus_turismo.txt, Palabras únicas: 153
Total de palabras únicas en los archivos .txt: 153


Eliminando tildes, signos de puntuación, mayusculas

In [ ]:
import unicodedata
import string

def contar_palabras_unicas(archivo):
    """
    Cuenta las palabras únicas en un archivo específico, eliminando tildes, signos de puntuación y mayúsculas.
    """
    unicas = set()
    with open(archivo, 'r', encoding='utf-8') as f:
        texto = f.read().lower()
        texto = unicodedata.normalize('NFD', texto)
        texto = ''.join(c for c in texto if unicodedata.category(c) != 'Mn') 
        texto = texto.translate(str.maketrans('', '', string.punctuation))    
        palabras = texto.split()
        unicas.update(palabras)
        print(f"{archivo}: {len(unicas)} únicas")
    return len(unicas)

archivo = r"C:\Users\ELI\Documents\RecInformación\ir25a-main\data\01_corpus_turismo.txt"
total = contar_palabras_unicas(archivo)
print("Total de palabras únicas (limpias):", total)

C:\Users\ELI\Documents\RecInformación\ir25a-main\data\01_corpus_turismo.txt: 145 únicas
Total de palabras únicas (limpias): 145


Tabla de palabras corpus pequeño, IDF

In [28]:
import os
from collections import defaultdict
import pandas as pd
import numpy as np
import unicodedata
import string

def limpiar(palabra):
    palabra = palabra.lower()
    palabra = unicodedata.normalize('NFD', palabra)
    palabra = ''.join(c for c in palabra if unicodedata.category(c) != 'Mn')
    palabra = palabra.translate(str.maketrans('', '', string.punctuation))
    return palabra

def generar_matriz_termino_documento(archivo):
    matriz = defaultdict(lambda: defaultdict(int))
    with open(archivo, 'r', encoding='utf-8') as f:
        lineas = f.readlines()
        total_documentos = len(lineas)
        for i, linea in enumerate(lineas):
            palabras = [limpiar(p) for p in linea.strip().split() if limpiar(p)]
            for palabra in palabras:
                matriz[palabra][f"Documento_{i+1}"] += 1

    df = pd.DataFrame(matriz).fillna(0).astype(int).T
    df.index.name = "Término"
    df.columns.name = "Documento"

    df['DF'] = (df > 0).sum(axis=1)

    #  IDF : log(N / df_t)
    df['IDF'] = df['DF'].apply(lambda df_t: np.log10(total_documentos / df_t) if df_t != 0 else 0)

    return df

# Ruta al archivo
archivo = r"C:\Users\ELI\Documents\RecInformación\ir25a-main\data\01_corpus_turismo.txt"
matriz_df = generar_matriz_termino_documento(archivo)

# Mostrar resultados
pd.set_option('display.max_rows', None)  
print("Matriz Término-Documento con IDF:")
print(matriz_df)


Matriz Término-Documento con IDF:
Documento       Documento_1  Documento_6  Documento_9  Documento_3  \
Término                                                              
ecuador                   1            0            0            0   
es                        1            1            1            0   
un                        1            0            1            1   
pais                      1            0            0            0   
megadiverso               1            0            0            0   
con                       1            0            0            1   
playas                    1            0            0            0   
montanas                  1            0            0            0   
y                         1            1            1            0   
selvas                    1            0            0            0   
ideales                   1            0            0            0   
para                      1            0            1   

In [23]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.5 MB 7.1 MB/s eta 0:00:02
   ---------- ----------------------------- 3.1/11.5 MB 8.0 MB/s eta 0:00:02
   ------------------- -------------------- 5.5/11.5 MB 9.1 MB/s eta 0:00:01
   ---------------------------- ----------- 8.1/11.5 MB 10.0 MB/s eta 0:00:01
   ------------------------------------- -- 10.7/11.5 MB 10.4 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 10.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   -------- ------------------------------- 2.6/12.6 MB 13.1 MB/s eta 0:00:01
   ----------------- ---------------------- 5.5/12.6 MB 13.3 MB/s eta 0:00:01
   ----------------------------- ---------- 9.2/12.6 MB 14.2 MB/s eta 0:00:01
   -------------------------------------- - 12.3/12.6 MB 14.5 MB/s eta 0:00:01
  


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\ELI\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
